<a href="https://colab.research.google.com/github/haripatel07/Smart_Agriculture_System/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
import os
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from werkzeug.utils import secure_filename

# Initialize Flask app
app = Flask(__name__)

# Configuration
UPLOAD_FOLDER = 'static/uploads/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# Define and load the model for plant disease classification
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
prediction = Dense(15, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)
model.load_weights('MobileNet_V2.h5')  # Load weights instead of the full model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Class names for plant diseases (replace with your actual class names)
class_names = [
    'class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7',
    'class8', 'class9', 'class10', 'class11', 'class12', 'class13', 'class14', 'class15'
]  # Update this with your 15 plant disease class names from train_data_generator.class_indices

# Function to check allowed file extensions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Function to preprocess the uploaded image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# Simple rule-based irrigation recommendation (placeholder)
def recommend_irrigation(temperature, humidity, soil_moisture):
    try:
        temp = float(temperature)
        hum = float(humidity)
        soil = float(soil_moisture)

        if soil < 30:  # Low soil moisture
            if temp > 30 and hum < 50:  # Hot and dry
                return "Irrigation highly recommended: Low soil moisture, high temperature, and low humidity."
            elif temp > 25:  # Warm
                return "Irrigation recommended: Low soil moisture and warm temperature."
            else:
                return "Irrigation suggested: Low soil moisture."
        elif soil < 50 and temp > 30:  # Moderate soil moisture, very hot
            return "Irrigation suggested: Moderate soil moisture but high temperature."
        else:
            return "No irrigation needed: Soil moisture is adequate."
    except ValueError:
        return "Invalid input: Please provide numeric values."

# Home route
@app.route('/', methods=['GET'])
def home():
    return app.send_static_file('index.html')  # Serve index.html from static folder

# Plant disease prediction route
@app.route('/predict_disease', methods=['POST'])
def predict_disease():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Preprocess the image
        processed_image = preprocess_image(filepath)

        # Make prediction
        predictions = model.predict(processed_image)
        predicted_class_idx = np.argmax(predictions, axis=1)[0]
        predicted_class = class_names[predicted_class_idx]
        confidence = float(np.max(predictions)) * 100

        # Return JSON response
        return jsonify({
            'prediction': f"{predicted_class} (Confidence: {confidence:.2f}%)"
        })
    else:
        return jsonify({'error': 'Invalid file format. Allowed formats: png, jpg, jpeg'}), 400

# Irrigation recommendation route
@app.route('/recommend_irrigation', methods=['POST'])
def recommend_irrigation_route():
    data = request.get_json()
    if not data or 'temperature' not in data or 'humidity' not in data or 'soil_moisture' not in data:
        return jsonify({'error': 'Missing required fields'}), 400

    temperature = data['temperature']
    humidity = data['humidity']
    soil_moisture = data['soil_moisture']

    recommendation = recommend_irrigation(temperature, humidity, soil_moisture)

    # Return JSON response
    return jsonify({'recommendation': recommendation})

# Run the app
if __name__ == '__main__':
    os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)  # Create upload folder if it doesn't exist
    app.static_folder = 'static'  # Set static folder
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
